In [ ]:
import numpy as np
from keras import layers, models, optimizers
from keras import backend as K
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from utils import combine_images
from PIL import Image
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
import seaborn as sns
sns.set()

#importint the necessary lirbraries

#import tensorflow.keras as kerasfrom __future__ import print_function

import keras
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



import tensorflow as tf
from keras import backend as K
from keras.layers import Layer
from keras import activations
from keras import utils
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.optimizers import Adam, SGD
from keras import optimizers
from keras.layers import *

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from os import listdir
import os
from imutils import paths
import numpy as np
import pickle
import matplotlib.pyplot as plt
import cv2
import random

K.set_image_data_format('channels_last')

In [ ]:
def CapsNet(input_shape, n_class, routings):
    """
    A Capsule Network on MNIST.
    :param input_shape: data shape, 3d, [width, height, channels]
    :param n_class: number of classes
    :param routings: number of routing iterations
    :return: Two Keras Models, the first one used for training, and the second one for evaluation.
            `eval_model` can also be used for training.
    """
    x = layers.Input(shape=input_shape)

    # Layer 1: Just a conventional Conv2D layer
    conv1 = layers.Conv2D(filters=128, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)

    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
    primarycaps = PrimaryCap(conv1, dim_capsule=8, n_channels=32, kernel_size=9, strides=2, padding='valid')

    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
                             name='digitcaps')(primarycaps)

    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    out_caps = Length(name='capsnet')(digitcaps)

    # Decoder network.
    y = layers.Input(shape=(n_class,))
    masked_by_y = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer. For training
    masked = Mask()(digitcaps)  # Mask using the capsule with maximal length. For prediction

    # Shared Decoder model in training and prediction
    decoder = models.Sequential(name='decoder')
    decoder.add(layers.Dense(512, activation='relu', input_dim=16*n_class))
    decoder.add(layers.Dense(1024, activation='relu'))
    decoder.add(layers.Dense(np.prod(input_shape), activation='sigmoid'))
    decoder.add(layers.Reshape(target_shape=input_shape, name='out_recon'))

    # Models for training and evaluation (prediction)
    train_model = models.Model([x, y], [out_caps, decoder(masked_by_y)])
    eval_model = models.Model(x, [out_caps, decoder(masked)])

    # manipulate model
    noise = layers.Input(shape=(n_class, 16))
    noised_digitcaps = layers.Add()([digitcaps, noise])
    masked_noised_y = Mask()([noised_digitcaps, y])
    manipulate_model = models.Model([x, y, noise], decoder(masked_noised_y))
    return train_model, eval_model, manipulate_model


def margin_loss(y_true, y_pred):
    """
    Margin loss for Eq.(4). When y_true[i, :] contains not just one `1`, this loss should work too. Not test it.
    :param y_true: [None, n_classes]
    :param y_pred: [None, num_capsule]
    :return: a scalar loss value.
    """
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

    return K.mean(K.sum(L, 1))

In [ ]:
#define the model
model, eval_model, manipulate_model = CapsNet(input_shape=(28, 28, 1),
                                                  n_class=10,
                                                  routings=3)
model.summary()

In [ ]:
#Now rewriting the algorithm to preprocess about 500 images in my own pipeline
#only do this if you have enough RAM
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir, 0)
        if image is not None:
            image = cv2.resize(image, (28, 28))
            return img_to_array(image)
        else:
            return np.array([])
        
    except Exception as e:
        print(f"Error : {e}")
        return None

image_list = []

directory_root = '../dataset/train'
imagePaths = list(paths.list_images(directory_root))
imagePaths = imagePaths[0:-1:2]
random.shuffle(imagePaths)
labels = [p.split(os.path.sep)[-2] for p in imagePaths]

batch_size = 16
try:
    for i in np.arange(0, len(imagePaths), batch_size):
        batchPaths = imagePaths[i:i + batch_size]
        batchLabels = labels[i: i + batch_size]
        
        for image in batchPaths:
            image_list.append(convert_image_to_array(image))
except Exception as e:
    print(f"Error: {e}")


len(image_list), len(labels)

np_image_list = np.array(image_list, dtype = np.float16) / 255.0
print(f"[INFO] splitting data")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, labels, test_size = 0.2, random_state = 42)

In [ ]:
np_image_list.shape

In [ ]:
y_test = LabelBinarizer().fit_transform(y_test)
y_train = LabelBinarizer().fit_transform(y_train)

In [ ]:
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
x_train.shape, x_test.shape

In [ ]:
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))
x_train.shape, x_test.shape

In [ ]:
y_test.shape

In [ ]:
x_train.shape, len(y_test)
classNames = [str(x) for x in np.unique(labels)]

In [ ]:
#fitting the model
model.compile(optimizer=optimizers.Adam(lr=0.001),
                  loss=[margin_loss, 'mse'],
                  loss_weights=[1., 0.392],
                  metrics={'capsnet': 'accuracy'})

weight_path = '../models/best_weights_capsule_train.h5'
checkpoint = ModelCheckpoint(weight_path, monitor='val_capsnet_acc',
                                           save_best_only=True, save_weights_only=True, verbose=1)
history = model.fit([x_train, y_train], [y_train, x_train], validation_data = [[x_test, y_test], [y_test, x_test]], epochs = 100, batch_size = 16, callbacks=[checkpoint])

In [ ]:
acc = history.history['capsnet_acc']
val_acc = history.history['val_capsnet_acc']
loss = history.history['capsnet_loss']
val_loss = history.history['val_capsnet_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.savefig('./Plots/Accuracy_Capsulenet_Greyscale.png')
plt.figure()
plt.show()
plt.clf()

In [ ]:
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.figure()
plt.show()
plt.savefig('./Plots/Loss_Capsulenet_Greyscale.png')
